In [23]:
import pandas as pd
import numpy as np

# 假设有1000个用户和20个商品
n_users = 10000
n_items = 10

# 创建用户和商品的ID
users = np.random.randint(0, n_users, size=10000)
items = np.random.randint(0, n_items, size=10000)

# 创建一个简单的用户行为数据集，这里仅用用户ID和商品ID
data = pd.DataFrame({'user_id': users, 'item_id': items})

# 分割训练集和测试集
train_data = data.sample(frac=0.8, random_state=123)
test_data = data.drop(train_data.index)


In [24]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding, Dense, Input, Flatten, Concatenate
from tensorflow.keras.models import Model

def get_din_model(n_users, n_items, embedding_size):
    # 输入层
    user_id_input = Input(shape=(1,), name='user_id')
    item_id_input = Input(shape=(1,), name='item_id')
    hist_item_input = Input(shape=(10,), name='hist_item_id') # 历史行为长度设为10

    # 嵌入层
    embedding_layer = Embedding(n_items, embedding_size, input_length=10, name='item_embedding')
    user_embedding = Embedding(n_users, embedding_size, input_length=1, name='user_embedding')

    # 用户和商品的嵌入
    user_embed = user_embedding(user_id_input)
    item_embed = embedding_layer(item_id_input)
    hist_item_embed = embedding_layer(hist_item_input)

    # DIN的注意力机制
    # 这里简化处理，仅用Flatten和Dense模拟
    hist_item_embed = Flatten()(hist_item_embed)
    user_embed = Flatten()(user_embed)
    item_embed = Flatten()(item_embed)

    attention_input = Concatenate()([hist_item_embed, user_embed, item_embed])
    attention_output = Dense(64, activation='relu')(attention_input)
    attention_output = Dense(1, activation='sigmoid')(attention_output)

    # 输出层
    output = Dense(1, activation='sigmoid')(attention_output)

    # 构建模型
    model = Model(inputs=[user_id_input, item_id_input, hist_item_input], outputs=output)
    return model

# 创建模型实例
embedding_size = 8
model = get_din_model(n_users, n_items, embedding_size)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [25]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 hist_item_id (InputLayer)      [(None, 10)]         0           []                               
                                                                                                  
 user_id (InputLayer)           [(None, 1)]          0           []                               
                                                                                                  
 item_id (InputLayer)           [(None, 1)]          0           []                               
                                                                                                  
 item_embedding (Embedding)     multiple             80          ['item_id[0][0]',                
                                                                  'hist_item_id[0][0]']     

In [26]:
# 准备训练数据
train_user_ids = np.array(train_data['user_id'])
train_item_ids = np.array(train_data['item_id'])
train_hist_items = np.random.randint(0, n_items, size=(len(train_data), 10))

# 随机生成标签
train_labels = np.random.randint(0, 2, size=(len(train_data), 1))

# 训练模型
model.fit([train_user_ids, train_item_ids, train_hist_items], train_labels, epochs=100, batch_size=32)


Epoch 1/100
250/250 [==============================] - 2s 1ms/step - loss: 0.6955 - accuracy: 0.5073
Epoch 2/100
250/250 [==============================] - 0s 1ms/step - loss: 0.6877 - accuracy: 0.5181
Epoch 3/100
250/250 [==============================] - 0s 1ms/step - loss: 0.6123 - accuracy: 0.7596
Epoch 4/100
250/250 [==============================] - 0s 1ms/step - loss: 0.5433 - accuracy: 0.8350
Epoch 5/100
250/250 [==============================] - 0s 1ms/step - loss: 0.5056 - accuracy: 0.8496
Epoch 6/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4750 - accuracy: 0.8612
Epoch 7/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4438 - accuracy: 0.8758
Epoch 8/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4175 - accuracy: 0.8863
Epoch 9/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3948 - accuracy: 0.8939
Epoch 10/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3756 - accura

250/250 [==============================] - 0s 1ms/step - loss: 0.2396 - accuracy: 0.9326
Epoch 82/100
250/250 [==============================] - 0s 1ms/step - loss: 0.2387 - accuracy: 0.9331
Epoch 83/100
250/250 [==============================] - 0s 1ms/step - loss: 0.2376 - accuracy: 0.9337
Epoch 84/100
250/250 [==============================] - 0s 1ms/step - loss: 0.2380 - accuracy: 0.9334
Epoch 85/100
250/250 [==============================] - 0s 1ms/step - loss: 0.2378 - accuracy: 0.9336
Epoch 86/100
250/250 [==============================] - 0s 1ms/step - loss: 0.2374 - accuracy: 0.9337
Epoch 87/100
250/250 [==============================] - 0s 1ms/step - loss: 0.2372 - accuracy: 0.9339
Epoch 88/100
250/250 [==============================] - 0s 2ms/step - loss: 0.2371 - accuracy: 0.9339
Epoch 89/100
250/250 [==============================] - 0s 1ms/step - loss: 0.2369 - accuracy: 0.9340
Epoch 90/100
250/250 [==============================] - 0s 1ms/step - loss: 0.2370 - accuracy: 

In [27]:
# 准备测试数据
test_user_ids = np.array(test_data['user_id'])
test_item_ids = np.array(test_data['item_id'])
test_hist_items = np.random.randint(0, n_items, size=(len(test_data), 10))

# 随机生成测试标签
test_labels = np.random.randint(0, 2, size=(len(test_data), 1))

# 评估模型
evaluation = model.evaluate([test_user_ids, test_item_ids, test_hist_items], test_labels)
print(f"Test Loss: {evaluation[0]}, Test Accuracy: {evaluation[1]}")


63/63 [==============================] - 0s 2ms/step - loss: 1.3913 - accuracy: 0.5060
Test Loss: 1.391257643699646, Test Accuracy: 0.5059999823570251


In [28]:

# 预测点击率
# 这里假设我们有一个包含所有项目ID的数组
item_ids = np.array(range(n_items)).reshape(-1, 1)
user_ids = np.array([test_user_ids[1]] * n_items)  # 某个特定用户的ID
hist_items = np.array([test_hist_items[1]] * n_items)  # 该用户的历史行为

# 预测每个项目的点击概率
click_probabilities = model.predict([user_ids, item_ids, hist_items])

# 输出结果
for item_id, prob in zip(item_ids, click_probabilities):
    print(f"Item ID {item_id}: Click Probability {prob}")


1/1 [==============================] - 0s 113ms/step
Item ID [0]: Click Probability [0.90674967]
Item ID [1]: Click Probability [0.9074385]
Item ID [2]: Click Probability [0.8561776]
Item ID [3]: Click Probability [0.90651387]
Item ID [4]: Click Probability [0.73678535]
Item ID [5]: Click Probability [0.26368397]
Item ID [6]: Click Probability [0.90434355]
Item ID [7]: Click Probability [0.897314]
Item ID [8]: Click Probability [0.90751076]
Item ID [9]: Click Probability [0.6645498]
